In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMolNotebooks
  ✓ CairoMakie
  ✓ MorphoMol
  ✓ MorphoMolNotebooks
  3 dependencies successfully precompiled in 114 seconds. 281 already precompiled.
[ Info: Precompiling MorphoMolNotebooks [3c5b58fc-d541-4bcd-a332-95372d8cdad8]


In [3]:
folder = "../../Data/selected_simulation_results/wip/"
@load "$(folder)simulation.jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [3]:
function generate_alpha_shape_persistence_diagrams(input, output, selection = nothing)
    if isnothing(selection)
        selection = 1:length(output["states"])
    end
    configurations = [MorphoMol.Utilities.get_point_vector_realization(state, input["template_centers"]) for state in output["states"][selection]]
    n_atoms_per_mol = length(input["template_radii"])

    as_dgms = [MorphoMol.Energies.get_alpha_shape_persistence_diagram(points) for points in configurations]
    alpha_shape_persistence_diagrams = Dict("diagrams" => [[dgms[1], dgms[2], dgms[3]] for dgms in as_dgms])
end

alpha_shape_persistence_diagrams = generate_alpha_shape_persistence_diagrams(input, output)
@save "$(folder)alpha_shape_persistence_diagrams.jld2" alpha_shape_persistence_diagrams

In [4]:
function generate_interfaces(input, output, selection = nothing)
    if isnothing(selection)
        selection = 1:length(output["states"])
    end
    configurations = [MorphoMol.Utilities.get_point_vector_realization(state, input["template_centers"]) for state in output["states"][selection]]
    n_atoms_per_mol = length(input["template_radii"])
    interfaces = [MorphoMol.Energies.get_interface_persistence_diagram_and_geometry(conf, n_atoms_per_mol) for conf in configurations]
    interface_diagrams = [interface[1] for interface in interfaces]
    interface_vertices = [interface[2] for interface in interfaces]
    interface_filtrations = [interface[3] for interface in interfaces]
    interfaces = Dict("diagrams" => interface_diagrams, "vertices" => interface_vertices, "filtrations" => interface_filtrations)
end

n = length(output["Es"])
e_mindex = argmin(output["Es"])
selection = maximum([25, e_mindex-500]):minimum([e_mindex + 500, n])
interfaces = generate_interfaces(input, output, selection)
@save "$(folder)interfaces.jld2" interfaces